# Assignment 4 
# Trading Strategy using Time-Series Forecast

---
In this assignment, we will develop a trading strategy using the time-series forecast techniques. 



I will help you develop idea by showing with simple ARIMA model method. 

1. First, we will download necessary library

In [1]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import itertools    
import warnings
import statsmodels.tsa.arima_model as arima_model
warnings.filterwarnings("ignore")
def arimaParameterSearch(data, p_max, d_max, q_max):
  """
  ARIMA Parameter search function
  The goal is to search for the phi and theta in the 
  ARIMA model where 
  data: price time series data
  p_max: the maximum number of autoregressive terms (number of phis)
  d_max: the maximum number of differences 
  q_max:the maximum number of moving-average terms (number of phis)
  """

  p = range(p_max + 1)
  d = range(d_max+ 1)
  q = range(q_max+ 1)
  parameter_spaces = list(itertools.product(p, d, q))
  best_aic, pdq= None, None
  for (p, d, q) in parameter_spaces:   
    try :
      model = arima_model.ARIMA(data, order=(p,d,q))
      result = model.fit(method="mle")
      if (best_aic is None) or (result.aic < best_aic):
        
        best_aic = result.aic
        pdq = (p,d,q)
        print(pdq, best_aic)
    except:
      continue
  return best_aic, pdq

In [8]:
from scipy.stats import norm 
def difference(data, interval=1):
  """
  create time different y[t] - y[t - interval]
  data: time-series data
  interval: time lag 
  Return 
      y[t] - y[t - interval]
  """
  return data - data.shift(interval)
def integrate(diff_y, hist_data, interval=1):
  """
  inverse version of time difference function
  diff_y: time-series of y[t] - y[t - interval]
  hist_data: time-series of y[t - interval]
  Return
    y[t] + y[t - interval]
  """
  return diff_y + hist_data.shift(interval)

def buildARIMAForecast(phis, theta, d, mu, sigma=0):
  """
  Create the ARIMA forecast function
  y[t] = mu + phi_1*y[t-1]+...+phi_p*y[t-p] + w[t] 
         + theta_1*w[t-1] + ...+theta_q * w[t-q]
  Inputs
    phis: array of autregressive parameters [phi_1, phi_2, .... phi_p]
    theta: array of moving average parameters [theta_1, ..., theta_q]
    d: difference order Ex: d=0, we work with y[t] = p[t], d=1, 
        we work with y[t] =p[t] - p[t-1]
    mu: mu parameter is the model
    sigma: standard deviation of forecasting error. 
  Return:
    ARIMA forecast function
  """

  p = len(phis)
  q = len(theta)
  def arimaForecast(data, alpha=0.95, res0=0):
    """
    arima 1-step forecast function
    y[t] = mu + phi_1*y[t-1]+...+phi_p*y[t-p] + w[t] 
         + theta_1*w[t-1] + ...+theta_q * w[t-q]
    Input:
      data: time-series data that we want to perform forecast
      alpha: where 1- alpha is confidence level 
      res0: initial residue. 
    Return:
      x_pred: 1-step predicted prices
      residue: forecast residue error.
      [conf_int_low, conf_int_high]: list containting lower and upper confidence intervals
    """
    x = data.copy()
    if d > 0: # d > 0, we work with differences
      for k in range(d): # difference orders
        x = difference(x, interval=1)
    x = x.dropna() # drop unusable data. 
    num_x = x.shape[0]
    pq_max = max(p,q) # maximum between autoregressive and moving average
    residue = np.zeros((num_x,))
    residue[:pq_max] = res0 # initialized residues
    x_out = [] # for storing forecast values
    dates = [] # for storing dates where we have forecast values 
    for k in range(pq_max, num_x): # for each day
      x_pred = mu # y_hat[t] = mu
      if p > 0:
        x_hist = (x.iloc[k-p:k] - mu) #y[t-1],.., y[t-p]
        x_pred += (phis * x_hist).sum()        
        # y_hat[t] = y_hat[t] + phi_1*y[t-1]+...+phi_p*y[t-p] + w[t] 
      if q > 0:
        w_hist = residue[k-q:k] #w[t], ..., w[t-q]
        x_pred += (theta * w_hist).sum()
        #y_hat[t] += y_hat[t] + theta_1*w[t-1] + ...+theta_q * w[t-q]     
      x_out.append(x_pred)
      dates.append(x.index[k])

      ## update residue 
      residue[k] = x.iloc[k] - x_pred #w[t] = y[t] -  y_hat[t]
    x_pred = pd.Series(x_out, index=dates) # Covert to time series
    residue = pd.Series(residue[pq_max:], index=dates) # Covert to time series
    if d > 0:
      for k in range(d):
        x_pred = integrate(x_pred, data).dropna()
    # make confidence intervale
    zalpha2 = norm.ppf(1 - alpha/2) # compute z_{alpha/2}
    conf_int_high = x_pred + zalpha2 * sigma # upper 95% confidence
    conf_int_low = x_pred - zalpha2 * sigma # lower 95% confidence 
    return x_pred, residue , [conf_int_low, conf_int_high]
  return arimaForecast 


In [9]:
try:
  theta = result.maparams
except:
  theta = np.array([])
try:
  phi = result.arparams
except:
  phi = np.array([])
mu = result.params['const']
d = pdq[1]
print(f"d: {d}")
print(r"thata: {}".format(theta))
print(r"phi: {}".format(phi))
print(f"Mu: {mu} ")
forecast_vol_sigma = result._forecast_error(steps=1)
print(f"1 Step forecast has STD: {forecast_vol_sigma}")

d: 0
thata: []
phi: [0.99719517]
Mu: 7.157378458639854 
1 Step forecast has STD: [0.00672737]


In [14]:
def buildARIMAIndicator(price,  confidence_interval, is_plot=False):
  """
  Trading using ARIMA Forecast:
  Inputs: 
          price: pandas time-series of the prices. 
          confidence_interval: list of two time-series of the lower and upper confidence values of predicted price
          is_plot: If true, this code will plot the Adj close price and show the buy and sell points.
  output:
          signal indicator: the series of indicators 1 to take the position, 0 for keep previous status and -1 to clear the position.

  """

  conf_low = confidence_interval[0]
  conf_high = confidence_interval[1]
  signal = []
  status = 0 #0==> no position, 1==>  buy all
  buy_price = 0
  for k in range(price.shape[0]):
    date = price.index[k]    
    if date in conf_low.index:      
      # only on the dates where we have confidence interval
      if (price[date] > conf_high[date]) and (status == 0): 
        # buy only when the close price is higher than upper part of the confidence
        # interval 
        # Basically, we are sure that the price siginificantly up.
        signal.append(1) # signal = 1 ==> buy
        status = 1
        buy_price = price[date]
        print(f"{date} Buy at {buy_price}.")

      elif (buy_price >  conf_low[date]) and (status == 1): 
        # sell only when the buy price is higher than higher part of the confidence
        # Basically, we are sure that the price really going down
        signal.append(-1) #signal = -1 ==> sell       
        status = 0   
        print(f"{date} Sell at {price[date]}")
      else:
        signal.append(0) # signal = 0 ==> do not thing.
    else:
      signal.append(0)

  signal = pd.Series(data=signal, index=price.index)
  buy_sig = price[signal == 1]
  sell_sig = price[signal == -1]
  if is_plot:
    plt.figure(figsize=(15,10))
    plt.plot(price)
    plt.plot(conf_high)
    plt.plot(conf_low) 
    plt.plot(buy_sig, color="red", marker="^",linestyle="")   
    plt.plot(sell_sig, color="black", marker="v",linestyle="")  
    plt.legend(["Adj Close","High", "Low", "Buy", "Sell"])
    plt.title("Bollinger Bands")
    plt.grid()
  return signal

In [16]:
def techicalTrading(price, # dataframe of a stock
                    indicator, # trade indicator
                    initial_money=1000000 #Starting money
                    ):
  """
  Trader simulation:
  Inputs: 
        price: time series of prices
        indicator: time-series indicator: 1 for Buy, -1 for sell, 0 for nothing
        initinal_money: initial money in the port. default 1M 
  output:
        wealth: time-series money in the port
  """

  num_dates = price.shape[0]
  status = "NONE" # Start with no poistion
  stock= 0
  money = initial_money
  wealth = []

  for k in range(num_dates):
    indicatork = indicator[k]
    if not np.isnan(indicatork): # if it is not NA
      if (indicatork > 0) and ( status == "NONE"):
        stock = money/price.iloc[k]
        #print(f"Buy at {close.values[k]}")
        money = 0
        status = "HOLD"
      elif (indicatork < 0) and ( status == "HOLD"):
        money = stock * price.iloc[k]
        stock =  0
        #print(f"Sell at {close.values[k]}")
        status = "NONE"
    cur_wealth = money + stock * price.iloc[k]
    wealth.append(cur_wealth)
  wealth = pd.Series(data=wealth, index=price.index)  
  return wealth 

Now, I want to you creat a new strategy where you will buy at the trough and sell at the peak. Here the trough between $t-k$ and $t$ is defined as  $T_{t-k,t} = \min(p[t], p[t-1], ..., p[t-l])$. Similarly, we define the peak between $t-k$ and $t$ as $P_{t-k,t} = \max(p[t], p[t-1], ..., p[t-l])$.

If we are is the state = 0, or no position from $t-k$ to $t$ we will buy an assest at its trough, or when $T_{t-k,t} < c_l[t]$ where $c_l[t]$ is the lower confidence value of the forecast. Similarly, if we are in the state = 1, or hold the assest, we will sell and assest at its peak, or when $P_{t-k,t} > c_h[t]$ where $c_h[t]$ is the upper confidence value of the forecast.

if $T_{t-k,t} < c_l[t]$, we have $(1-\alpha/2)\%$ that the trough is really the trough, and if $P_{t-k,t} > c_h[t]$, we have $(1-\alpha/2)\%$ that the peak is really the peak.

In [ ]:
#create a new indictor function here.
def buildARIMAIndicator(price, confidence_interval, is_plot=False):
  """
  Trading using ARIMA Forecast:
  Inputs: 
          price: pandas time-series of the prices. 
          confidence_interval: list of two time-series of the lower and upper confidence values of predicted price
          is_plot: If true, this code will plot the Adj close price and show the buy and sell points.
  output:
          signal indicator: the series of indicators 1 to take the position, 0 for keep previous status and -1 to clear the position.

  """

  conf_low = confidence_interval[0]
  conf_high = confidence_interval[1]
  signal = []
  status = 0 #0==> no position, 1==>  buy all
  buy_price = 0
  for k in range(price.shape[0]):
    date = price.index[k]    
    if date in conf_low.index:      
      # only on the dates where we have confidence interval
      if (price[date] > conf_high[date]) and (status == 0): 
        # buy only when the close price is higher than upper part of the confidence
        # interval 
        # Basically, we are sure that the price siginificantly up.
        signal.append(1) # signal = 1 ==> buy
        status = 1
        buy_price = price[date]
        print(f"{date} Buy at {buy_price}.")

      elif (buy_price >  conf_low[date]) and (status == 1): 
        # sell only when the buy price is higher than higher part of the confidence
        # Basically, we are sure that the price really going down
        signal.append(-1) #signal = -1 ==> sell       
        status = 0   
        print(f"{date} Sell at {price[date]}")
      else:
        signal.append(0) # signal = 0 ==> do not thing.
    else:
      signal.append(0)

  signal = pd.Series(data=signal, index=price.index)
  buy_sig = price[signal == 1]
  sell_sig = price[signal == -1]
  if is_plot:
    plt.figure(figsize=(15,10))
    plt.plot(price)
    plt.plot(conf_high)
    plt.plot(conf_low) 
    plt.plot(buy_sig, color="red", marker="^",linestyle="")   
    plt.plot(sell_sig, color="black", marker="v",linestyle="")  
    plt.legend(["Adj Close","High", "Low", "Buy", "Sell"])
    plt.title("Bollinger Bands")
    plt.grid()
  return signal

Determine the final wealth with train samples 

In [ ]:
# your code is here
indicator_train = buildARIMAIndicator(train_gold, confidence_int, is_plot=True)

gold_price_train = np.exp(train_gold) # convert back the acutal price
wealth_train = techicalTrading(gold_price_train, indicator_train, initial_money=1000000)
indicator_buy_hold = indicator_train *0 
indicator_buy_hold[0] = 1
wealth_buy_and_hold = techicalTrading(gold_price_train, indicator_buy_hold, initial_money=1000000)
plt.figure(figsize=(15,10))
plt.plot(wealth_train)
plt.plot(wealth_buy_and_hold)
plt.grid()
plt.legend(["ARIMA Model", "Buy and Hold"])

Determine the final wealth with train samples 

In [ ]:
# your code is here.
print(f"Final wealth using ARIMA: {wealth_train.iloc[-1]:,.2f}")

Next, we want to construct the volatility model using ARCH(1) on the residue 
Thus, we have $Y_t = \mu_t + r_t$  where
$\mu_t$ is the forecast using ARIMA model that we used earlier, and
$r_t$ is the residue. 

First we need to install arch

In [24]:
!pip install arch 

     |████████████████████████████████| 798kB 7.4MB/s 


In [29]:
from arch import arch_model
resid = result.resid
vol_model = arch_model(resid, mean="Constant", p=1, q=0, vol="arch")
result_vol = vol_model.fit()
print(result_vol.summary())



Iteration:      1,   Func. Count:      5,   Neg. LLF: -6482.685693543018
Iteration:      2,   Func. Count:     18,   Neg. LLF: -6482.689211329767
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -6482.689228486683
            Iterations: 2
            Function evaluations: 26
            Gradient evaluations: 2
                      Constant Mean - ARCH Model Results                      
Dep. Variable:                   None   R-squared:                      -0.000
Mean Model:             Constant Mean   Adj. R-squared:                 -0.000
Vol Model:                       ARCH   Log-Likelihood:                6482.69
Distribution:                  Normal   AIC:                          -12959.4
Method:            Maximum Likelihood   BIC:                          -12942.9
                                        No. Observations:                 1825
Date:                Thu, Feb 04 2021   Df Residuals:                     1822
Time:        

Now, please write the garch volatility forecast

$r_t = a_t$ \\
$\sigma_t^2 = \omega + \alpha\times\sigma_{t-1}^2 



In [ ]:
# your conde is here 
# look at the example in buildARIMAForecast 

def buildARCHVolForecast(omega, alpha):
   """
  Create the GARCH forecast function
  r[t] = a[t]
  sigma[t]^2 = omega + alpha * sigma[t-1]^2 
  omega and alpha are parameters for GARCH(1,1)
  Return:
    GARCH forecast function
  """
  def archForecast(y_arima, residue, confidence):
    """
      ARCH(1) forecasts
      Input:
      y_arima: time-series forecast from ARIMA
      residue: time-series forecast residue error 
      confidence: confidence level
    Return:
      x_pred: 1-step predicted prices
      sigma2: variance of forecast
      [conf_int_low, conf_int_high]: list containting lower and upper confidence intervals
    """
    x_pred = y_arima 
    # add your code here. Please note that x_pred = y_arima. 
    
    x = data.copy()
    if d > 0: # d > 0, we work with differences
      for k in range(d): # difference orders
        x = difference(x, interval=1)
    x = x.dropna() # drop unusable data. 
    num_x = x.shape[0]
    pq_max = max(p,q) # maximum between autoregressive and moving average
    residue = np.zeros((num_x,))
    residue[:pq_max] = res0 # initialized residues
    x_out = [] # for storing forecast values
    dates = [] # for storing dates where we have forecast values 
    for k in range(pq_max, num_x): # for each day
      x_pred = mu # y_hat[t] = mu
      if p > 0:
        x_hist = (x.iloc[k-p:k] - mu) #y[t-1],.., y[t-p]
        x_pred += (phis * x_hist).sum()        
        # y_hat[t] = y_hat[t] + phi_1*y[t-1]+...+phi_p*y[t-p] + w[t] 
      if q > 0:
        w_hist = residue[k-q:k] #w[t], ..., w[t-q]
        x_pred += (theta * w_hist).sum()
        #y_hat[t] += y_hat[t] + theta_1*w[t-1] + ...+theta_q * w[t-q]     
      x_out.append(x_pred)
      dates.append(x.index[k])

      ## update residue 
      residue[k] = x.iloc[k] - x_pred #w[t] = y[t] -  y_hat[t]
    x_pred = pd.Series(x_out, index=dates) # Covert to time series
    residue = pd.Series(residue[pq_max:], index=dates) # Covert to time series
    if d > 0:
      for k in range(d):
        x_pred = integrate(x_pred, data).dropna()
    # make confidence intervale
    zalpha2 = norm.ppf(1 - alpha/2) # compute z_{alpha/2}
    conf_int_high = x_pred + zalpha2 * sigma # upper 95% confidence
    conf_int_low = x_pred - zalpha2 * sigma # lower 95% confidence 
    return x_pred, residue , [conf_int_low, conf_int_high]
  return arimaForecast 
  




Use this new confidence interval to trade using the peak, and trough strategy. 
Is there any differences in profit for confidence=0.99? zalpha2 = norm.ppf(confidence)

In [ ]:
# your code is here.
pred_gold_test, resid_test, confidence_int_test = arima_gold(test_gold, alpha=0.01) # 99% Confidence
plt.figure(figsize=(15,10))
plt.plot(pred_gold_test)
plt.plot(test_gold)
plt.grid()
plt.legend(["Predicted value", "actual value"])

Lastly, let us try with difference confidence levels
Find wealths for confidence levels equal to 0.8 and 0.9999

In [ ]:
# your code is here. 
pred_gold_test, resid_test, confidence_int_test = arima_gold(test_gold, alpha=0.01) # 99% Confidence
plt.figure(figsize=(15,10))
plt.plot(pred_gold_test)
plt.plot(test_gold)
plt.grid()
plt.legend(["Predicted value", "actual value"])

Which confidence level give you the maximum wealth for
1. trains samples
2. test samples 

In [ ]:
# you code is here
gold_price_test = np.exp(test_gold) # convert back the acutal price
wealth_test = techicalTrading(gold_price_test, indicator_test, initial_money=1000000)
indicator_buy_hold_test = indicator_test *0 
indicator_buy_hold[0] = 1
wealth_buy_and_hold_test = techicalTrading(gold_price_test, indicator_buy_hold, initial_money=1000000)
plt.figure(figsize=(15,10))
plt.plot(wealth_test)
plt.plot(wealth_buy_and_hold_test)
plt.grid()
plt.legend(["ARIMA Model", "Buy and Hold"])
plt.title("Wealth of Test set")

In [ ]:
print(f"Final wealth using Buy and Hold: {wealth_buy_and_hold_test.iloc[-1]:,.2f}")
print(f"Final wealth using ARIMA: {wealth_test.iloc[-1]:,.2f}")